In [ ]:
!nvidia-smi

In [ ]:
import os
import sys
import warnings
from pprint import pprint
from glob import glob
from tqdm.notebook import tqdm
import pickle
import glob

from sklearn.linear_model import Ridge
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
sys.path.append('../input/pythonbox')
from box import Box
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
#from pytorch_grad_cam import GradCAMPlusPlus
#from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
from fastai.vision.all import *

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

#sys.path.append('../input/poolformer-master')
#import models as PoolFormerModels #poolformer

print(pl.__version__)
warnings.filterwarnings("ignore")
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

sys.path.append(f'../input/petfinder-scripts')


class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        dataset,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._dataset = dataset
        self._cfg = cfg

    def __create_dataset(self, train=True):
        return (
            self._dataset(self._test_df, self._cfg.transform.image_size)
            if train
            else self._dataset(self._test_df, self._cfg.transform.image_size)
        )

    def test_dataloader(self):
        dataset = self.__create_dataset(True)
        self._cfg.val_loader.batch_size = 32
        self._cfg.val_loader.num_workers = os.cpu_count()
        return DataLoader(dataset, **self._cfg.val_loader)

def transform_get_image_shape_describe(input_df): 
    pathes = input_df['Id'].values
    shapes = []
    for path in tqdm(pathes):
        img = Image.open(path)
        height, width = img.height, img.width
        shapes.append([height, width])
    input_df[["height", "width"]] = shapes
    input_df["aspect"] = input_df["height"] / input_df["width"]
    height_mean = 904.2843018563358
    height_std = 156.90598049629264
    width_mean = 804.4262510088781
    width_std = 270.21192072081044
    if len(test_df)==8:#testだと全部同じ大きさで標準化ミスるからエラー除去でいれる。
        input_df['height'] = input_df['height'].fillna(0)
        input_df['width'] = input_df['width'].fillna(0)
    else:
        input_df['height'] = input_df['height'].apply(lambda x: (x-height_mean)/ height_std)
        input_df['width'] = input_df['width'].apply(lambda x: (x-width_mean)/ width_std)

    print(f'height_mean{height_mean}_std{height_std}')
    print(f'width_mean{width_mean}_std{width_std}')
    return input_df
    
def get_predict_feature(model, test_loader, transforms,device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images, features in test_loader:
        images = images.to(device)
        features = features.to(device).float()
        images = transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images, features).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

def get_predict(model, test_loader, transforms,device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images in test_loader:
        images = images.to(device)
        images = transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

def model_load(model, exp_name, model_path):
    if exp_name == 'exp092' or exp_name == 'exp100':
        model.load_state_dict(torch.load(model_path)['state_dict'])
    else:
        model.load_state_dict(torch.load(model_path))
    return model

import exp108

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df["Id"] = test_df["Id"].apply(lambda x: os.path.join('../input/petfinder-pawpularity-score', "test", x + ".jpg"))
test_df = transform_get_image_shape_describe(test_df)

IMG_PREDICTS = []
exp_name = 'exp108'
print(exp_name)
config = eval(exp_name).config
config.model.pretrain = False
dataloader = PetfinderDataModule(test_df, 
                                 eval(exp_name).PetfinderDataset, 
                                 config).test_dataloader()

tmp = []
for fold in tqdm(range(config.n_splits)):
    model = eval(exp_name).Model(config)
    model_path = glob.glob(os.path.join('../input/petfinder-exp108', f'{exp_name}/fold{fold}/best_loss_fold{fold}.*'))[0]
    model.load_state_dict(torch.load(model_path)['state_dict'])
    predicts = get_predict_feature(model, dataloader, eval(exp_name).get_default_transforms, 'cuda')

    tmp.append(predicts)

    del model
    torch.cuda.empty_cache()
IMG_PREDICTS = tmp

#predict make
preds = []
fold_num = len(IMG_PREDICTS)
for fold in range(fold_num):
    preds.append(IMG_PREDICTS[fold] / fold_num)
predict = np.sum(preds, 0)
predict

In [ ]:
predictions = predict
submission = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission['Pawpularity'] = predictions
submission[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)
display(submission[['Id', 'Pawpularity']].head(8))